In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

- 2018년 train
- 2019 1월, 2월 test

- 치킨, 기온, 강수, 미세먼지 데이터 이용
- 요일별 EDA 해보고 '주말여부' 컬럼 추가 결정!

In [2]:
chicken2018 = pd.read_csv('chicken_data/CALL_CHICKEN_2018_01MONTH.csv')
for i in ['02','03','04','05','06','07','08','09','10','11','12']:
    a = 'chicken_data/CALL_CHICKEN_2018_' + i + 'MONTH.csv'
    b = pd.read_csv(a)
    chicken2018 = pd.concat([chicken2018,b])

In [3]:
chicken2017 = pd.read_csv('chicken_data/CALL_CHICKEN_2017_01MONTH.csv')
for i in ['02','03','04','05','06','07','08','09','10','11','12']:
    a = 'chicken_data/CALL_CHICKEN_2017_' + i + 'MONTH.csv'
    b = pd.read_csv(a)
    chicken2017 = pd.concat([chicken2017,b])

In [4]:
chicken2016 = pd.read_csv('chicken_data/CALL_CHICKEN_2016_01MONTH.csv')
for i in ['02','03','04','05','06','07','08','09','10','11','12']:
    a = 'chicken_data/CALL_CHICKEN_2016_' + i + 'MONTH.csv'
    b = pd.read_csv(a)
    chicken2016 = pd.concat([chicken2016,b])

In [5]:
chicken2019 = pd.read_csv('chicken_data/CALL_CHICKEN_2019_01MONTH.csv')
for i in ['02']:
    a = 'chicken_data/CALL_CHICKEN_2019_' + i + 'MONTH.csv'
    b = pd.read_csv(a)
    chicken2019 = pd.concat([chicken2019,b])

In [6]:
chicken_data = pd.concat([chicken2016,chicken2017,chicken2018,chicken2019])

In [8]:
chicken_data.to_csv('chicken_data.csv', index = False) 

# 치킨 데이터

In [2]:
chicken_data = pd.read_csv('chicken_data.csv', encoding = 'euc-kr')

In [3]:
chicken_data_서대문구 = chicken_data[lambda x : x.시군구 == '서대문구']
chicken_groupby = chicken_data_서대문구.groupby(['기준일','읍면동','요일'])['통화건수'].sum()

# 날씨 데이터
- 2018년도랑 2019년도 나누기
- 년도 split해서 분류하면 될듯 

In [13]:
air = pd.read_csv('air.csv')
dust_1617 = pd.read_csv('weather/dust_1617.csv')

In [14]:
air = air[lambda x : x.측정소명 == '서대문구']
air = air[['측정일자','미세먼지','초미세먼지']].reset_index(drop = True)

In [15]:
air = pd.concat([air, dust_1617])

In [17]:
weather_161718 = pd.read_csv('weather/weather_161718.csv', encoding = 'euc-kr')
weather_test = pd.read_csv('weather/test.weather.csv', encoding = 'euc-kr')

In [18]:
weather = weather_161718.fillna(0)
weather = pd.concat([weather_161718,weather_test])

In [19]:
# 연도
def date_year(x):
    return str(x)[:4]

def date_month(x):
    return str(x)[4:6]

def date_day(x):
    return str(x)[6:]

In [20]:
def date_year2(x):
    return x.split('-')[0]
def date_month2(x):
    return x.split('-')[1]
def date_day2(x):
    return x.split('-')[2]

In [21]:
air = air.groupby(['측정일자'])['미세먼지','초미세먼지'].mean().reset_index()

In [24]:
chicken_groupby['년'] = chicken_groupby['기준일'].map(date_year)
chicken_groupby['월']= chicken_groupby['기준일'].map(date_month)
chicken_groupby['일']= chicken_groupby['기준일'].map(date_day)

air['년'] = air['측정일자'].map(date_year)
air['월'] = air['측정일자'].map(date_month)
air['일'] = air['측정일자'].map(date_day)

weather['년'] = weather['일시'].map(date_year2)
weather['월'] = weather['일시'].map(date_month2)
weather['일'] = weather['일시'].map(date_day2)

In [27]:
use_col = ['읍면동', '요일', '통화건수', '년', '월', '일','미세먼지', '초미세먼지',  '평균기온(°C)', 
           '최저기온(°C)', '최고기온(°C)', '일강수량(mm)']

In [28]:
final_df = pd.merge(chicken_groupby,air , how = 'inner', on = ['년','월','일'])
final_df = pd.merge(final_df, weather , how = 'outer', on = ['년','월','일'])

In [29]:
final_df = final_df[use_col]

In [30]:
len(final_df)

8615

In [25]:
fill_mean_func = lambda x : x.fillna(x.mean())
final_df = final_df.groupby('월').apply(fill_mean_func)
final_df = final_df.reset_index(drop = True)
final_df.isnull().sum()

In [33]:
final_df = final_df.dropna()

In [34]:
final_test = final_df[lambda x : x.년 == '2019']
final_train = final_df[lambda x : x.년.isin(['2016','2017','2018'])]

In [35]:
final_train.to_csv('train.csv', index = False)
final_test.to_csv('test.csv', index = False)

In [23]:
final_train = pd.read_csv('train.csv', encoding = 'euc-kr')
final_test = pd.read_csv('test.csv', encoding = 'euc-kr')

In [26]:
soccer = pd.read_csv('soccer.csv')

In [27]:
def year(x):
    return str(x)[:4]
def month(x):
    return str(x)[5:6]
def day(x):
    return str(x)[7:]

In [28]:
soccer['년'] = soccer.날짜.map(year)
soccer['월'] = soccer.날짜.map(month)
soccer['일'] = soccer.날짜.map(day)

In [29]:
soccer['soccer'] = 1

In [30]:
day_map = {'월':0, '화':0, '수':0, '목':0, '금':1, '토':1, '일':1}
final_train['weekend'] = final_train['요일'].map(day_map)

In [31]:
area_map = {'홍제동':3, '연희동':4, '대현동':3, '남가좌동':2, '북가좌동':2, '북아현동':1, '합동':1,'홍은동':2,'창천동':3,'충정로2가':4,
          '충정로3가':1,'봉원동':1,'신촌동':1,'현저동':1,'대신동':1}
final_train['읍면동'] = final_train['읍면동'].map(area_map)

In [32]:
final_test['weekend'] = final_test['요일'].map(day_map)
final_test['읍면동'] = final_test['읍면동'].map(area_map)

In [35]:
final_train2 = final_train.groupby(['읍면동','년','월','일']).mean().reset_index()

In [36]:
soccer.년 =  soccer.년.map(int)
soccer.월 =  soccer.월.map(int)
soccer.일 =  soccer.일.map(int)

final_train2.년 =  final_train2.년.map(int)
# train_1.월 =  train_1.월.map(int)
# train_1.일 =  train_1.일.map(int)

# test_1.년 =  test_1.년.map(int)
# test_1.월 =  test_1.월.map(int)
# test_1.일 =  test_1.일.map(int)

In [37]:
final_train3 = pd.merge(final_train2, soccer, on =['년','월','일'],how = 'left')
# test_1 = pd.merge(test_1, soccer, on =['년','월','일'],how = 'left')

In [38]:
del final_train3['날짜']
final_train3 = final_train3.fillna(0)

In [76]:
# del test_1['날짜']
# test_1 = test_1.fillna(0)

In [46]:
temp = pd.get_dummies(final_train3['읍면동'],prefix = '읍면동')
final_train4 = pd.concat([temp, final_train3], axis = 1)

In [48]:
final_train4.to_csv('for_sejung.csv', index = False)

In [63]:
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
# from sklearn.decomposition import PCA

C:\anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


StandardScaling을 하였다.

In [ ]:
dfX1 = for_sejung [['미세먼지','평균기온(°C)','일강수량(mm)']]
scaler1 = StandardScaler().fit(dfX1)
dfX1=pd.DataFrame(scaler1.transform(dfX1), columns = ['미세먼지','평균기온(°C)','일강수량(mm)'])
dfX1['weekend'] = for_sejung ['weekend'].reset_index(drop = True)
dfX1['soccer'] = for_sejung ['soccer'].reset_index(drop = True)
dfX1['읍면동_1'] = for_sejung ['읍면동_1'].reset_index(drop = True)
dfX1['읍면동_2'] = for_sejung ['읍면동_2'].reset_index(drop = True)
dfX1['읍면동_3'] = for_sejung ['읍면동_3'].reset_index(drop = True)

dfX1 = sm.add_constant(dfX1)
dfy1 = for_sejung.통화건수.reset_index(drop=True)

In [78]:
glm1 = sm.GLM(dfy1, dfX1, family = sm.families.Poisson()).fit()

In [79]:
print(glm1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   통화건수   No. Observations:                  885
Model:                            GLM   Df Residuals:                      879
Model Family:                 Poisson   Df Model:                            5
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -4149.1
Date:                Mon, 20 May 2019   Deviance:                       3954.9
Time:                        17:29:34   Pearson chi2:                 4.23e+03
No. Iterations:                     4                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0791      0.010    320.907      0.000       3.060       3.098
미세먼지           0.0410      0.007      5.984      0.0